Apuração da Premiação Tina (Individual)



In [40]:
import pandas as pd
import numpy as np
from pathlib import Path

In [41]:
pd.set_option('display.max_columns', None)

In [42]:
# Dia da apuração
# TODO TROCAR A DATA PARA O ANALISTA DE NEGÓCIOS RODAR NO DIA DA APURAÇÃO
# hoje = pd.to_datetime('today').date()
hoje = pd.to_datetime('2022-05-27')
ANO=str(hoje.year)

In [43]:
# DIA PADRÃO DE APURAÇÃO
#DIA_APURA=27
DIA_APURA=hoje.day

In [44]:
DIA_APURA

27

In [45]:
# SELECIONANDO EMPRESA
ID_UNN=1

dict_EMPRESAS={1:"TELHANORTE", 4:"TUMELERO"}
empresa=dict_EMPRESAS[ID_UNN]
print(ID_UNN)
print(empresa)

1
TELHANORTE


In [46]:
MESES= ['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
I_MESES= ['01','02','03','04','05','06','07','08','09','10','11','12']
MES=MESES[hoje.month-1]
I_MES=I_MESES[MESES.index(MES)]
I_MES_ANTERIOR=I_MESES[MESES.index(MES)-1]


In [47]:
I_MES_ANTERIOR

'04'

In [48]:
# Período de Apuração   -   Faltas injustificadas no período
# 27 de Janeiro de 2022
DATA_I = pd.to_datetime('2022-'+I_MES_ANTERIOR+'-'+str(DIA_APURA))  # data inicial AAAA-MM-DD
DATA_F = pd.to_datetime('2022-'+I_MES+'-'+str(DIA_APURA-1))  # data final AAAA-MM-DD

In [49]:
print(DATA_I)
print(DATA_F)

2022-04-27 00:00:00
2022-05-26 00:00:00


In [50]:
# PASTAS 

DIR_BASE='C:/apuracao/'
DIR_INFO=DIR_BASE+'info/'
DIR_TINA_BASES=DIR_INFO
DIR_ENTRA=DIR_BASE+'ENTRADA/'
# PASTA PARA MANTER AS BASES ORIGINAIS
DIR_IN=DIR_BASE+ANO+'/'+MES+'/ORIGINAIS/'
# PASTA COM PLANILHA DE CHECAGEM PARA O ANALISTA DE NEGÓCIOS
DIRETORIO_CHECAGEM=DIR_BASE+ANO+'/'+MES+'/CHECAGEM/'
# PASTA PARA PLANILHAS DAS VERBAS PARA RH 
DIR_RH=DIR_BASE+ANO+"/"+MES+"/"+"para_RH/"



In [51]:
# CRIANDO PASTAS DE SAÍDA SE NÃO EXISTIREM
Path(DIR_IN).mkdir(parents=True, exist_ok=True)
Path(DIRETORIO_CHECAGEM).mkdir(parents=True, exist_ok=True)
Path(DIR_RH).mkdir(parents=True, exist_ok=True)


In [52]:
# Arquivos a serem importados
# Incluídas lojas TUMELERO
F_ALVO = DIR_INFO+"target_salesperson.csv" # 
F_VERBAS = DIR_INFO+"/verbas_descricao.xlsx"

F_LOJA_SAFRA=DIR_INFO+"lojas.csv" # 
#f_loja_safra = "info/lojas_pilotos_safra3.xlsx" # Exemplo de janeiro


In [65]:

# Ricardo Valdir via Teams para o Luis no dia da apuração
#f_comissao = dir_in+"LANCCOMISSAO022022.csv"
E_COMISSAO = DIR_ENTRA+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+empresa.lower()
F_COMISSAO = DIR_IN+"LANC_COMISSAO_"+str(I_MES)+ANO+'_'+empresa.lower()

# Vitor (Wingg) via e-mail no dia da apuração
E_EXTRATO = DIR_ENTRA+"extract_remuneration_"+ANO+I_MES+str(DIA_APURA)+".xlsx"
F_EXTRATO = DIR_IN+"extract_remuneration_"+ANO+I_MES+str(DIA_APURA)+".xlsx"

# Thayna via sharepoint antes da apuração
# F_FALTAS = DIR_IN+"FALTAS_INJUSTIFICADAS_PREMIACOES TINA_23mar2022_ThaynaCarneiro.xlsx"
E_FALTAS = DIR_ENTRA+"PREMIACOES TINA.xlsx"
F_FALTAS = DIR_IN+"PREMIACOES TINA.xlsx"

In [54]:
print(F_COMISSAO)

C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_telhanorte


In [55]:
print('A planilha de checagem da Apuração da PREMIAÇÃO TINA INDIVIDUAL realizada em '+MES+' está aqui:')
print(DIRETORIO_CHECAGEM)
# ARQUIO DE SAÍDA PARA CHECAGEM DO ANALISTA DE NEGÓCIOS - EM EXCEL
F_SAI_CHECAGEM=DIRETORIO_CHECAGEM + "checagem_apuração_PREMIACAO_INDIVIDUAL_"+MES+".xlsx"
print(F_SAI_CHECAGEM)


A planilha de checagem da Apuração da PREMIAÇÃO TINA INDIVIDUAL realizada em mai está aqui:
C:/apuracao/2022/mai/CHECAGEM/
C:/apuracao/2022/mai/CHECAGEM/checagem_apuração_PREMIACAO_INDIVIDUAL_mai.xlsx


In [56]:
# Sufixos possíveis
sufixo="_sem_considerar_faltas_injustificadas"
#sufixo="_analista"

In [57]:
# Verbas para o RH das premiações Tina Individuais
F_VERBA_ASS=DIR_RH+"VERBA_4118_SEGMENTO_ASSESSORES_"+empresa+".xlsx"
F_VERBA_VEN_REN=DIR_RH+"VERBA_4119_RENTABILIDADE_VENDAS_"+empresa+".xlsx"
F_VERBA_VEN_DEC=DIR_RH+"VERBA_4120_DECORACAO_VENDAS_"+empresa+".xlsx"
F_VERBA_VEN_SUP=DIR_RH+"VERBA_4121_SUPERACAO_VENDAS_"+empresa+".xlsx"
F_VERBA_VEN_TEC=DIR_RH+"VERBA_4122_AREAS_TECNICAS_VENDAS_"+empresa+".xlsx"


In [58]:
# Verbas para o RH das premiações Tina Individuais SEM CONSIDERAR FALTAS INJUSTIFICADAS
F_VERBA_ASS_SEM=DIR_RH+"VERBA_4118_SEGMENTO_ASSESSORES_"+empresa+sufixo+".xlsx"
F_VERBA_VEN_REN_SEM=DIR_RH+"VERBA_4119_RENTABILIDADE_VENDAS_"+empresa+sufixo+".xlsx"
F_VERBA_VEN_DEC_SEM=DIR_RH+"VERBA_4120_DECORACAO_VENDAS_"+empresa+sufixo+".xlsx"
F_VERBA_VEN_SUP_SEM=DIR_RH+"VERBA_4121_SUPERACAO_VENDAS_"+empresa+sufixo+".xlsx"
F_VERBA_VEN_TEC_SEM=DIR_RH+"VERBA_4122_AREAS_TECNICAS_VENDAS_"+empresa+sufixo+".xlsx"

In [59]:
F_VERBA_VEN_REN

'C:/apuracao/2022/mai/para_RH/VERBA_4119_RENTABILIDADE_VENDAS_TELHANORTE.xlsx'

In [60]:
# Parâmetros do Modelo
# Modelo Vendedores
# Modelo Vendedores Comissão
gat_min_comissao = 0.0  # Gatilho minimo comissão
ANG_COEF_COMISSAO = 0.01  # Coeficiente angular = 1%
lin_coef_comissao = 0.0  # Coeficiente Linear = 0.0%

# Modelo Vendedores Comissão Turbinada Decoração
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
gat_min_decoracao = 0.0  # Gatilho minimo decoração
ANG_COEF_DECORACAO = 0.02  # Coeficiente angular decoração = 2%
lin_coef_decoracao = 0.0  # Coeficiente Linear decoração = 0.0%

# Modelo Vendedores Comissão turbinada Áreas Técnicas
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
gat_min_areastecnicas = 0.0  # Gatilho minimo Áreas Técnicas = 0.0%
ANG_COEF_AREASTECNICAS= 0.005  # Coeficiente angular Áreas Técnicas = 0.5%
lin_coef_areastecnicas = 0.0  # Coeficiente Linear decoração = 0.0%

# Modelo Vendedores Prêmio superação vendas
M1 = 100  # meta1 = 100%
M2 = 150  # meta2 = 150%
M3 = 200  # meta3 = 200%
M4 = 250  # meta4 = 250%
B1 = 300  # bonus1 = R$300,00 -> ganha se bater a meta1
B2 = 600  # bonus2 = R$600,00 -> ganha se bater a meta2
B3 = 900  # bonus3 = R$900,00 -> ganha se bater a meta3
B4 = 1200  # bonus4 = R$1200,00 -> ganha se bater a meta4

# Modelo Vendedores Prêmio Rentabilidade
GAT_MIN_RENTAB = 100.  # Tem que bater 100% da rentabilidade
ANG_COEF_RENTAB = 0.001  # Coeficiente angular da rentabilidade = 0.1%
LIN_COEF_RENTAB = -0.1  # Coeficiente linear da rentabilidade = -0.1
KICKER = 200.


In [61]:
# Parâmetros do Modelo
# Modelo Assessores
ASS_A1 = 30000  # Alvo1 = R$30.000,00
ASS_A2 = 50000  # Alvo2 = R$50.000,00
ASS_B1 = 100  # Bônus1 caso bata Alvo1
ASS_B2 = 200  # Bônus2 caso bata Alvo2


In [64]:
nome_lojas = pd.read_csv(F_LOJA_SAFRA, encoding="ISO-8859-1", delimiter=";")

In [66]:
import shutil
# Fazendo backup das bases nas pasta ORIGINAIS respectiva ao mes e ano da apuração 
shutil.copy2(E_COMISSAO,F_COMISSAO)
shutil.copy2(E_EXTRATO,F_EXTRATO)
shutil.copy2(E_FALTAS,F_FALTAS)

'C:/apuracao/2022/mai/ORIGINAIS/PREMIACOES TINA.xlsx'

In [67]:
# importando as comissões
print("Importando as comissões")
print("Arquivo: "+F_COMISSAO)
column_names=[
    'N1',	
    'N2',	
    'N3',	
    'MATRICULA',	
    'POS_NEG',	 
    'VALOR',
    'NONE'
    ]
comissao = pd.read_csv(F_COMISSAO, delimiter=";", names=column_names, header=None, skiprows=1)

Importando as comissões
Arquivo: C:/apuracao/2022/mai/ORIGINAIS/LANC_COMISSAO_052022_telhanorte


In [68]:
# Trocando "," por "." para poder calcular os números no Python
comissao["VALOR"]=comissao["VALOR"].str.replace(",",".").astype(float)


In [69]:
##  Vendas devolvidas como negativas
print("Valores negativos para vendas devolvidas")

comissao["VALOR_COR"]=np.where(comissao["POS_NEG"]==3771, -comissao["VALOR"],comissao["VALOR"])


Valores negativos para vendas devolvidas


In [70]:
# Só matrícula e valor das vendas correspondente a essa matrícula
comissao=comissao[['MATRICULA','VALOR_COR']]

In [71]:
# Somando as vendas totais de cada matrícula
# TODO nomear origem dos arquivos 
print("Somando todas as vendas de uma matrícula:")
comissao = comissao.groupby(['MATRICULA'])['VALOR_COR'].sum().reset_index()
comissao.rename(columns={'VALOR_COR':'VENDAS_TOT'}, inplace=True)
comissao['COMISSAO']=comissao['VENDAS_TOT']*0.01


Somando todas as vendas de uma matrícula:


In [72]:
# Para visualizar as vendas totais e a comissão de um colaborador 
comissao[comissao['MATRICULA'] == 28586]

,MATRICULA,VENDAS_TOT,COMISSAO
798,28586,-4748.04,-47.4804


In [101]:
# Lendo os alvos que os vendedores devem atingir por loja
print("Lendo alvos das lojas no arquivo:")
print(F_ALVO)
alvos = pd.read_csv(F_ALVO, delimiter=";", header=0, skiprows=0)


Lendo alvos das lojas no arquivo:
C:/apuracao/info/target_salesperson.csv


In [102]:

alvos.rename(columns={'target_sales_salesperson':'Alvo1'}, inplace=True)
alvos['Alvo1']=alvos['Alvo1'].str.replace(",",".").astype(float)

In [103]:
alvos['Minimo Garantido']=alvos['floor_remuneration'].str.replace(",",".").astype(float)


In [104]:
# Mantendo somente alvos Telhanorte
alvos=alvos[alvos['id_unn']==1] # id_unn = (1=Telhanorte; 2=Tumelero)

In [105]:
# Mantendo somente alguns parâmetros
alvos= alvos[['cod_loja','Alvo1', 'Minimo Garantido']]

In [106]:
alvos_telha=pd.merge(alvos,nome_lojas[['cod_loja','sigla_loja']],how='left',on='cod_loja')

In [107]:
# Dicionário de alvo por codigo da loja
dict_alvo=dict(zip(alvos_telha['sigla_loja'], alvos_telha['Alvo1']))

In [108]:
# Dicionário de alvo por codigo da loja
dict_minimo_garantido=dict(zip(alvos_telha['sigla_loja'], alvos_telha['Minimo Garantido']))

In [109]:
# Lendo os alvos que os vendedores devem atingir por loja
#print("Lendo alvos das lojas no arquivo:")
# print(F_ALVO)
# alvos = pd.read_excel(F_ALVO, header=0, skiprows=1)
# alvos['Minimo Garantido']=alvos['Minimo Garantido']/100.
#a[a['Loja']==7992]
# alvos.columns

In [110]:
# Lendo faltas injustificadas
print("Lendo faltas injustificadas no arquivo:")
print(F_FALTAS)
faltas_inj = pd.read_excel(F_FALTAS, header=0)

Lendo faltas injustificadas no arquivo:
C:/apuracao/TELHANORTE/2022/mai/ORIGINAIS/PREMIACOES TINA.xlsx


In [111]:
faltas = faltas_inj[(faltas_inj['DATA DA OCORRÊNCIA'] >= DATA_I) &
           (faltas_inj['DATA DA OCORRÊNCIA'] <= DATA_F)]
faltas['DATA DA OCORRÊNCIA'].sort_values()

9     2022-04-27
35    2022-04-27
151   2022-04-27
224   2022-04-27
244   2022-04-27
         ...    
75    2022-05-15
91    2022-05-15
99    2022-05-15
138   2022-05-15
74    2022-05-15
Name: DATA DA OCORRÊNCIA, Length: 196, dtype: datetime64[ns]

In [112]:
# Loading Remuneration extract from Tina
print("Lendo Extrato de Remuneração da Tina")
print(F_EXTRATO)
extrato = pd.read_excel(F_EXTRATO)

Lendo Extrato de Remuneração da Tina
C:/apuracao/TELHANORTE/2022/mai/ORIGINAIS/extract_remuneration_20220527.xlsx


In [113]:
# Somente colaboradores Tina das lojas da safra
print("Considerando somente colaboradores das lojas participantes da Tina")
COLS_TO_KEEP=[
    'store_sk', 
    'registration_id', 
    'segment', 
    'job_simplified',
    'E1_gross_sales', 
    'E2_gross_sales', 
    'E3_gross_sales', 
    'gross_sales', 
    'sales_segment', 
    'rentability'
    ]
so_safra = extrato[extrato["store_sk"].isin(nome_lojas['cod_loja'])][COLS_TO_KEEP].copy(deep=True)
so_safra.columns

Considerando somente colaboradores das lojas participantes da Tina


Index(['store_sk', 'registration_id', 'segment', 'job_simplified',
       'E1_gross_sales', 'E2_gross_sales', 'E3_gross_sales', 'gross_sales',
       'sales_segment', 'rentability'],
      dtype='object')

In [114]:
#para planilha checagem
tina=so_safra.copy(deep=True)


In [115]:
# Verificando quem tem falta injustificada
tina['FALTA']=np.where(tina['registration_id'].isin(faltas['MATR EMPR'])==True,'sim','nao')

In [116]:
# Juntando nome de sigla das lojas
tina=pd.merge(
    tina.rename(columns={'store_sk':'cod_loja'}),
    nome_lojas[nome_lojas['id_unn']==1][['cod_loja', 'sigla_loja']],
    how='left',on='cod_loja'
    )

In [117]:
# Renomeando matricula para juntar com a comissão
tina.rename(columns={'registration_id':'MATRICULA'}, inplace=True)


In [118]:
# # Juntando com a comissão
tina=tina.merge(comissao,how='left',on='MATRICULA')

In [119]:

tina=pd.merge(
    tina,
    alvos_telha[['sigla_loja','Alvo1', 'Minimo Garantido']],
    how='left', on='sigla_loja'
    )

In [120]:
tina['vendas_sobre_alvo']=tina['VENDAS_TOT']/tina['Alvo1']

In [121]:
# Eliminando alvos de assessores
tina['Alvo1']=np.where(tina['job_simplified']=='assistant',np.nan,tina['Alvo1'])

In [122]:
# Eliminando minimos garantidos de assessores
tina['Minimo Garantido']=np.where(tina['job_simplified']=='assistant',np.nan,tina['Minimo Garantido'])

In [123]:
# Eliminando vendas sobre alvo de assessores
tina['vendas_sobre_alvo']=np.where(tina['job_simplified']=='assistant',np.nan,tina['vendas_sobre_alvo'])

In [124]:
tina['recebe_premiacao']=np.where(tina['vendas_sobre_alvo']>=1.,'sim','nao')

In [125]:
#Salvando verba
# Entradas: v verba, n nome_verba, x numero_verba, f f_nome
def salva_verba(v,x,f):
#    v.rename(columns={ v.columns[0]: 'MATRICULA'}, inplace=True)
    v.rename(columns={ v.columns[1]: "VALOR" }, inplace=True)
    v['ZERO'] = 0
    v['VERBA'] = x
    v=v[['MATRICULA', 'ZERO', 'VERBA','VALOR']]
    v = v[v['VALOR'] > 0]
    # Saving as a excel file for RH
    writer = pd.ExcelWriter(f, engine='xlsxwriter')  # #
    v.to_excel(writer, sheet_name="VERBA "+str(x), index=False)
    writer.save()



In [126]:
def alvo_bonus_rentabilidade(vendas_tot,rentabilidade,vendas_seg,alvo):
#     # Modelo Vendedores Prêmio Rentabilidade
#    T E S T A D O     E     C H E C A D O
# gat_min_rentab = 100.  # Tem que bater 100% da rentabilidade
# ang_coef_rentab = 0.001  # Coeficiente angular da rentabilidade = 0.1%
# lin_coef_rentab = -0.1  # Coeficiente linear da rentabilidade = -0.1
# kicker = 200.
    global GAT_MIN_RENTAB, ANG_COEF_RENTAB, LIN_COEF_RENTAB, KICKER
    if vendas_tot>=alvo and rentabilidade >= GAT_MIN_RENTAB:
    #    # TODO DELETAR SE OK
#        val_rent = KICKER DELETAR SE OK
        # TODO DELETAR SE OK
        #if (rentabilidade-GAT_MIN_RENTAB)>0: DELETAR SE OK
        val_rent=(rentabilidade*ANG_COEF_RENTAB+LIN_COEF_RENTAB)*vendas_seg+KICKER
            # val_rent=val_rent+int(r-gat_min_rentab)*ang_coef_rentab*s
    else:
        val_rent=0.0
    return val_rent



In [127]:
# Calculando rentabilidade
tina['REM_REN'] = np.where(
    tina['job_simplified']=='salesperson',
    tina.apply(
    lambda x: alvo_bonus_rentabilidade(x['VENDAS_TOT'],x['rentability'],x['sales_segment'],x['Alvo1']), axis=1
    ).copy(),np.nan
    )


In [128]:
# Salva verba rentabilidade 
salva_verba(tina[['MATRICULA','REM_REN']].copy(deep=True),4119,F_VERBA_VEN_REN_SEM)
salva_verba(tina[['MATRICULA','REM_REN']][tina['FALTA']=='nao'].copy(deep=True),4119,F_VERBA_VEN_REN)


In [129]:
# Calculating bonus for SALESPERSON
# Verba 4120 - Segmento vendedores - DECORAÇÃO
def alvo_bonus_decoracao(vendas_seg,vendas_tot,alvo):
# Modelo Vendedores Prêmio DECORAÇÃO vendas
#    T E S T A D O     E     C H E C A D O
# Modelo Vendedores Comissão Turbinada Decoração
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
# gat_min_decoracao = 0.0  # Gatilho minimo decoração
# ang_coef_decoracao = 0.02  # Coeficiente angular decoração = 2%
# lin_coef_decoracao = 0.0  # Coeficiente Linear decoração = 0.0%

    global ANG_COEF_DECORACAO 
    if vendas_tot>=alvo:
        bonus = vendas_seg*ANG_COEF_DECORACAO
    else:
        bonus=0.0
    return bonus


In [130]:
# Calculando Verba DECORAÇÃO 
tina['DECORACAO'] = np.where(
    tina['job_simplified']=='salesperson',
    tina.apply(
    lambda x: alvo_bonus_decoracao(x['E2_gross_sales'],x['VENDAS_TOT'],x['Alvo1']), axis=1).copy(),np.nan
    )



In [131]:
# SALVA verba DECORAÇÃO     
salva_verba(tina[['MATRICULA','DECORACAO']].copy(deep=True),4120,F_VERBA_VEN_DEC_SEM)
salva_verba(tina[['MATRICULA','DECORACAO']][tina['FALTA']=='nao'].copy(deep=True),4120,F_VERBA_VEN_DEC)


In [132]:
# Calculating bonus for SALESPERSON
def alvo_bonus_superacao(vendas_tot):
# Modelo Vendedores Prêmio superação vendas
# T E S T A D O    E    C H E C A D O 
# m1 = 100  # meta1 = 100%
# m2 = 150  # meta2 = 150%
# m3 = 200  # meta3 = 200%
# m4 = 250  # meta4 = 250%
# b1 = 300  # bonus1 = R$300,00 -> ganha se bater a meta1
# b2 = 600  # bonus2 = R$600,00 -> ganha se bater a meta2
# b3 = 900  # bonus3 = R$900,00 -> ganha se bater a meta3
# b4 = 1200  # bonus4 = R$1200,00 -> ganha se bater a meta4
    global M1, M2, M3, M4, B1, B2, B3, B4 # metas e respectivos bonus
    if vendas_tot>=M4:
        bonus = B4
    elif vendas_tot>=M3:
        bonus = B3
    elif vendas_tot>=M2:
        bonus= B2
    elif vendas_tot>=M1:
        bonus=B1
    else:
        bonus=0.0
    return bonus



In [133]:
# Calculando Prêmio SUPERAÇÃO   
tina['SUPERACAO'] = np.where(
    tina['job_simplified']=='salesperson',
    tina.apply(
    lambda x: alvo_bonus_superacao(x['VENDAS_TOT']/x['Alvo1']*100.), axis=1).copy(),np.nan
    )

In [134]:
# SALVA verba SUPERAÇÃO    
salva_verba(tina[['MATRICULA','SUPERACAO']].copy(deep=True),4121,F_VERBA_VEN_SUP_SEM)
salva_verba(tina[['MATRICULA','SUPERACAO']][tina['FALTA']=='nao'].copy(deep=True),4121,F_VERBA_VEN_SUP)


In [135]:
def alvo_bonus_areastecnicas(vendas_seg,vendas_tot,alvo):
# Modelo Vendedores Prêmio ÁREAS TÉCNICAS
#    T E S T A D O     E     C H E C A D O
# Modelo Vendedores Comissão  ÁREAS TÉCNICAS
# Como as safras 1, 2 e 3 tem os mesmos valores, mantive só um valor para todas as safras
# gat_min_areastecnicas = 0.0  # Gatilho minimo Áreas Técnicas = 0.0%
# ang_coef_areastecnicas = 0.005  # Coeficiente angular Áreas Técnicas = 0.5%
# lin_coef_areastecnicas = 0.0  # Coeficiente Linear decoração = 0.0%

    global ANG_COEF_AREASTECNICAS # gat_min_areastecnicas, ang_coef_areastecnicas, lin_coef_areastecnicas 
    if vendas_tot>=alvo:
        bonus = vendas_seg*ANG_COEF_AREASTECNICAS
    else:
        bonus=0.0
    return bonus




In [136]:
# Calculando Premiação Individual ÁREAS TÉCNICAS
tina['AREAS_TEC'] = np.where(
    tina['job_simplified']=='salesperson',
    tina.apply(
    lambda x: alvo_bonus_areastecnicas(x['E1_gross_sales'],x['VENDAS_TOT'],x['Alvo1']), axis=1).copy(),np.nan
    )

In [137]:
# SALVA verba PREMIAÇÃO ÁREAS TÉCNICAS
salva_verba(tina[['MATRICULA','AREAS_TEC']].copy(deep=True),4122,F_VERBA_VEN_TEC_SEM)
salva_verba(tina[['MATRICULA','AREAS_TEC']][tina['FALTA']=='nao'].copy(deep=True),4122,F_VERBA_VEN_TEC)


In [138]:
# Calculando Prêmio Assessor
def alvo_bonus_assessor(vendas_seg):
    global ASS_A1, ASS_A2, ASS_B1, ASS_B2 # ass_a1, ass_a2, ass_b1, ass_b2
    if vendas_seg >= ASS_A2:
        bonus = ASS_B2
    elif vendas_seg >= ASS_A1:
        bonus = ASS_B1
    else:
        bonus = 0.0
    return bonus
    

In [139]:
tina.columns

Index(['cod_loja', 'MATRICULA', 'segment', 'job_simplified', 'E1_gross_sales',
       'E2_gross_sales', 'E3_gross_sales', 'gross_sales', 'sales_segment',
       'rentability', 'FALTA', 'sigla_loja', 'VENDAS_TOT', 'COMISSAO', 'Alvo1',
       'Minimo Garantido', 'vendas_sobre_alvo', 'recebe_premiacao', 'REM_REN',
       'DECORACAO', 'SUPERACAO', 'AREAS_TEC'],
      dtype='object')

In [140]:
# Somente assessores
tina['PREMIO_ASSESSOR'] = np.where(
    tina['job_simplified']=='assistant',
    tina.apply(
    lambda x: alvo_bonus_assessor(x['sales_segment']), axis=1).copy(),np.nan
    )

    

In [141]:
tina['diferenca_vendas_Tina_ADP']=tina['gross_sales']-tina['VENDAS_TOT']

In [142]:
tina[tina['MATRICULA']==3343]

,cod_loja,MATRICULA,segment,job_simplified,E1_gross_sales,E2_gross_sales,E3_gross_sales,gross_sales,sales_segment,rentability,FALTA,sigla_loja,VENDAS_TOT,COMISSAO,Alvo1,Minimo Garantido,vendas_sobre_alvo,recebe_premiacao,REM_REN,DECORACAO,SUPERACAO,AREAS_TEC,PREMIO_ASSESSOR,diferenca_vendas_Tina_ADP
17,30.0,3343.0,E3,salesperson,51685.504,3622.766,337217.547,393523.817,337217.547,103.067833,nao,MOR,393523.81,3935.2381,270000.0,1985.0,1.457496,sim,1234.52703,72.45532,300.0,258.42752,NaN,0.007


In [143]:
tina.columns

Index(['cod_loja', 'MATRICULA', 'segment', 'job_simplified', 'E1_gross_sales',
       'E2_gross_sales', 'E3_gross_sales', 'gross_sales', 'sales_segment',
       'rentability', 'FALTA', 'sigla_loja', 'VENDAS_TOT', 'COMISSAO', 'Alvo1',
       'Minimo Garantido', 'vendas_sobre_alvo', 'recebe_premiacao', 'REM_REN',
       'DECORACAO', 'SUPERACAO', 'AREAS_TEC', 'PREMIO_ASSESSOR',
       'diferenca_vendas_Tina_ADP'],
      dtype='object')

In [144]:
# SALVA verba PREMIAÇÃO ASSESSOR
salva_verba(tina[['MATRICULA','PREMIO_ASSESSOR']].copy(deep=True),4118,F_VERBA_ASS_SEM)


In [145]:
salva_verba(tina[['MATRICULA','PREMIO_ASSESSOR']][tina['FALTA']=='nao'].copy(deep=True),4118,F_VERBA_ASS)


In [146]:
tina.to_excel(F_SAI_CHECAGEM)